In [1]:
import os
os.chdir('..')

import json
from workflows.skeletons import *
from utils import visualize, os_lib

In [2]:
def log(title, content):
    if isinstance(content, dict):
        content = json.dumps(content, indent=4, ensure_ascii=False)
    elif isinstance(content, list):
        _content = ''
        for c in content:
            _content += str(c) + '\n'
        content = _content

    print(
        f"{visualize.TextVisualize.highlight_str(title)}\n"
        f"==========\n"
        f"{content}\n"
        f"----------\n"
    )

In [3]:
from models.bailian import Model

class LLM(Model):
    def on_process_end(self, obj, debug=False, **kwargs):
        if debug:
            log('模型输出', obj['content'])
        return obj
    
# export API_KEY='xxx'
llm = LLM(
    api_key=os.environ.get('API_KEY'),
    # llm_type='qwen2-7b-instruct'
)

In [4]:
os.environ['LANG'] = 'ch'
from templates.make_sentences import *

In [5]:
class MakeExtendClsPrompt(Module):
    def on_process(self, obj, debug=False, **kwargs):
        prompt = make_skeleton(
            additional_prompts=[
                make_role('经验丰富的深度学习训练师'),
                make_encourage(),
                make_task(
                    task='构建意图识别训练集',
                    desc='补充"关于某种疾病的定义"的问题分类，要求问句的文法尽量多变，尽量口语化，构造的句子有长有短，不要出现重复句式。至少列出10条。'
                        '\n语料模板中，疾病名称用{disease}代替，每个句子以`<definition>`开头。\n',
                ),
                make_examples([
                    '<definition>什么是{disease}?',
                    '<definition>{disease}是什么意思?',
                    '<definition>能解释一下{disease}这个病吗?'
                ]),
            ]
        )

        if debug:
            log('模型输入', prompt)

        obj.update(post_kwargs=dict(prompt=prompt))
        return obj

pipeline = Pipeline(
    MakeExtendClsPrompt(),
    llm,
    # 自行在这后面添加结果解析、入库等流程
)
obj = pipeline({}, debug=True)

模型输入

现在你扮演一名经验丰富的深度学习训练师。请尽你所能去回答问题。
现在需要完成一个"构建意图识别训练集"的任务，这个关于这个任务的详细描述：
补充"关于某种疾病的定义"的问题分类，要求问句的文法尽量多变，尽量口语化，构造的句子有长有短，不要出现重复句式。至少列出10条。
语料模板中，疾病名称用{disease}代替，每个句子以`<definition>`开头。

这是一些样例，样例文本使用`<example>`和`</example>`包裹起来。

<example>
<definition>什么是{disease}?
<definition>{disease}是什么意思?
<definition>能解释一下{disease}这个病吗?
</example>

开始!


----------

模型输出
<definition>我听说过{disease}，但具体是啥病呢？
<definition>能不能通俗点讲讲{disease}是怎么回事？
<definition>{disease}这个名词听起来挺专业的，它究竟指的是什么疾病？
<definition>如果有人得了{disease}，意味着他们身体出了什么问题？
<definition>我想了解一下{disease}的医学定义，可以告诉我吗？
<definition>怎样才算被诊断为{disease}？它的界定标准是什么？
<definition>{disease}听起来挺复杂的，能不能简单描述一下它的特征？
<definition>对于普通人来说，怎么理解{disease}这个病症比较容易？
<definition>我好奇{disease}是不是就是常说的那种...它的正式定义是什么？
<definition>医生提到的{disease}，是指的哪一类健康状况？
----------



In [6]:
class MakeClsPrompt(Module):
    def on_process(self, obj, debug=False, **kwargs):
        query = obj['query']
        prompt = make_skeleton(
            query=make_query(query),
            additional_prompts=[
                make_role('医生'),
                make_encourage(),
                make_task(
                    task='意图识别',
                    desc='病人会询问一些问题，请了解病人问这个问题的目的，识别出病人的潜在意图。'
                ),
                make_access_cls(['疾病定义', '疾病病因', '治疗方法']),
                make_examples([
                    ['感冒是什么？', '疾病定义'],
                    ['为什么会引起感冒？', '疾病病因'],
                    ['感冒要吃什么药？', '治疗方法']
                ]),
            ]
        )

        if debug:
            log('模型输入', prompt)

        obj.update(post_kwargs=dict(prompt=prompt))
        return obj

pipeline = Pipeline(
    MakeClsPrompt(),
    llm,
)
obj = {'query': '怎么突然就失眠了？'}
obj = pipeline(obj, debug=True)

模型输入

现在你扮演一名医生。请尽你所能去回答问题。
现在需要完成一个"意图识别"的任务，这个关于这个任务的详细描述：
病人会询问一些问题，请了解病人问这个问题的目的，识别出病人的潜在意图。
这是一些可选的分类项:
["疾病定义", "疾病病因", "治疗方法"]

这是一些样例，样例文本使用`<example>`和`</example>`包裹起来。

<example>

Question: 感冒是什么？
Answer: 疾病定义

Question: 为什么会引起感冒？
Answer: 疾病病因

Question: 感冒要吃什么药？
Answer: 治疗方法
</example>

开始!

Question: 怎么突然就失眠了？


----------

模型输出
疾病病因
----------

